**PART 0**

**AUTHOR FILKIN MAKSIM**

---



Подключим библиотеки

In [0]:
import torchvision
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms
import time
import sklearn.metrics as metrics
import seaborn as sn
import pandas as pd

Загрузим данные

In [0]:
opt = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
best_res = 0

Функция для тренировки

In [0]:
def train():
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        if (batch_idx % 100 == 0):
            print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
               % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

Функция для теста

In [0]:
def test():
  global best_res
  correct = 0
  total = 0
  with torch.no_grad():
      for data in testloader:
          images, labels = data
          images, labels = images.to(device), labels.to(device)
          outputs = net(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
  if (best_res < 100. * correct / total):
      best_res = 100. * correct / total
  print('Accuracy of the network on the test images: %.3f %%' % (
          100. * correct / total))

Функция для построения confusion table

In [0]:
def confusion_table(netw):
    class_correct = np.zeros((10,10))
    with torch.no_grad():
      for data in testloader:
          images, labels = data
          images, labels = images.to(device), labels.to(device)
          outputs = netw(images)
          _, predicted = torch.max(outputs.data, 1)
          correct = (predicted == labels).squeeze()
          for i in range(len(labels)):
              class_correct[labels[i], predicted[i]] += 1
    class_correct = class_correct / class_correct.sum(axis=1, keepdims=True)
    df_cm = pd.DataFrame(class_correct, index = classes, columns = classes)
    plt.figure(figsize = (10,7))
    sn.heatmap(df_cm, annot=True)
    

Функция для построения precision, recall, f1



In [0]:
def precision_recall_f1(netw):
    y_true = np.array(testset.targets)
    y_pred = []
    with torch.no_grad():
      for data in testloader:
          images, labels = data
          images, labels = images.to(device), labels.to(device)
          outputs = netw(images)
          _, predicted = torch.max(outputs.data, 1)
          for i in range(len(labels)):
              y_pred.append(int(predicted[i]))
    y_pred = np.array(y_pred)
    precision, recall, f1, _ = metrics.precision_recall_fscore_support(y_true, y_pred,labels=list(range(10)))
    prcsn_rcll_f1 = [precision,recall,f1]
    df_cm = pd.DataFrame(prcsn_rcll_f1, index = ['precision','recall','F1'], columns = classes)
    plt.figure(figsize = (10,7))
    sn.heatmap(df_cm, annot=True)

**TASK 1**

---



Класс начальной модели

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #32 32
        self.conv1 = nn.Conv2d(3, 6, 5)
        #28 28
        self.pool = nn.MaxPool2d(2, 2)
        #14 14
        self.conv2 = nn.Conv2d(6, 16, 5)
        #10 10 
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Сеть и оптимизатор начальной модели

In [0]:
best_res = 0
net = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

main function

In [0]:
for __ in range(0,2):
    print("epoh =",__ + 1)
    train()
    test()
print('Best: %.3f %%' % (best_res))

In [0]:
confusion_table(net)

In [0]:
precision_recall_f1(net)

**TASK 2**

---





Решил изменить фотографии

In [0]:
opt = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Класс модели

In [0]:
def temp(pre_val, next_val):
    return [nn.Conv2d(pre_val, next_val, kernel_size=3, padding=1), nn.BatchNorm2d(next_val), nn.ReLU(inplace=True)]

class Better_net(nn.Module):
    def __init__(self):
        super(Better_net, self).__init__()
        features = []
        features += temp(3,64)
        features += temp(64,64)
        features += [nn.MaxPool2d(2,2)]
        features += temp(64,128)
        features += temp(128,128)
        features += [nn.MaxPool2d(2,2)]
        features += temp(128,256)
        features += temp(256,256)
        features += temp(256,256)
        features += [nn.MaxPool2d(2,2)]
        features += temp(256,512)
        features += temp(512,512)
        features += temp(512,512)
        features += [nn.MaxPool2d(2,2)]
        features += temp(512,512)
        features += temp(512,512)
        features += temp(512,512)
        features += [nn.MaxPool2d(2,2)]
        features += [nn.AvgPool2d(1,1)]
        self.features = nn.Sequential(*features)
        self.classifier = nn.Linear(512, 10)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

*Создаем* сеть и SGD

In [0]:
best_res = 0
net = Better_net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
#optimizer = torch.optim.Adam(net.parameters(),lr = 1e-4)

main function

In [0]:
for __ in range(0,200):
    print("epoh =",__ + 1)
    train()
    test()
print('Best: %.3f %%' % (best_res))

**TASK 3**

---



In [0]:
confusion_table(net)

In [0]:
precision_recall_f1(net)